In [52]:
import pandas as pd
import json
import math

In [53]:
def get_pythagoras(x, y):
    return math.sqrt(x ** 2 + y ** 2)
diffPath = 'accsaber-maps/362bb (Novocaine - Kassi)/ExpertPlusStandard.dat' #Replace with your difficulty file's path
infoPath = 'accsaber-maps/362bb (Novocaine - Kassi)/Info.dat' #Replace with your info file's path
with open(diffPath) as diff_json_data:
    diffData = json.load(diff_json_data)

with open(infoPath) as info_json_data:
    infoData = json.load(info_json_data)

In [54]:
#ADD IMPLEMENTATION FOR NO BPM CHANGES LATER



#Parse BPM changes if they exist
initialBPM = infoData.get('_beatsPerMinute')
bpmChangesDict = diffData.get('_customData').get('_BPMChanges')
df_BPMChanges = pd.DataFrame(bpmChangesDict)
df_BPMChanges

,_time,_BPM,_beatsPerBar,_metronomeOffset
0,5.938,156,4,0


In [55]:
df = pd.DataFrame(diffData['_notes'])
df['_yCenter'] = df.loc[:, ('_lineLayer')].apply(lambda x: 1 + x * 0.55)
df['_xCenter'] = df.loc[:, ('_lineIndex')].apply(lambda x: -0.9 + x * 0.6)

#Add bpm column
df['_bpm'] = initialBPM
for i in range(len(df)):
    currentTime = df.loc[i, '_time']
    currentRow = 0
    for j in range(len(df_BPMChanges)):
        if currentTime >= df_BPMChanges.loc[j, '_time']:
            df['_bpm'] = df_BPMChanges.loc[j, '_BPM']
   

        
left = (df[df['_type'] == 0]) #All left handed notes
right = (df[df['_type'] == 1]) #All right handed notes

left['_xMovement'] = left.loc[:, ['_xCenter']].diff().fillna(0)
left['_yMovement'] = left.loc[:, ['_yCenter']].diff().fillna(0)
left['_totMovement'] = left.apply(lambda x: get_pythagoras(x['_xMovement'], x['_yMovement']), axis=1).fillna(0) 
left['_angleChange'] = left.apply(lambda x: math.atan(x['_yMovement']/x['_xMovement']), axis=1)
left['_timeChange'] = left.loc[:, ['_time']].diff().fillna(0)

right['_xMovement'] = right.loc[:, ['_xCenter']].diff().fillna(0)
right['_yMovement'] = right.loc[:, ['_yCenter']].diff().fillna(0)
right['_totMovement'] = right.apply(lambda x: get_pythagoras(x['_xMovement'], x['_yMovement']), axis=1).fillna(0)
right['_angleChange'] = right.apply(lambda x: math.atan(x['_yMovement']/x['_xMovement']), axis=1)
right['_timeChange'] = right.loc[:, ['_time']].diff().fillna(0)

average_angle = (left['_angleChange'].mean() + right['_angleChange'].mean()) / 2

left['_seconds'] = (60 * left['_timeChange']) / left['_bpm']
right['_seconds'] = (60 * right['_timeChange']) / right['_bpm']

/var/folders/hx/8pb__z8x49q872rhs7wcd7_m0000gn/T/ipykernel_13678/2931912547.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left['_xMovement'] = left.loc[:, ['_xCenter']].diff().fillna(0)
/var/folders/hx/8pb__z8x49q872rhs7wcd7_m0000gn/T/ipykernel_13678/2931912547.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left['_yMovement'] = left.loc[:, ['_yCenter']].diff().fillna(0)
/var/folders/hx/8pb__z8x49q872rhs7wcd7_m0000gn/T/ipykernel_13678/2931912547.py:21: SettingWithCopyWarning: 
A value is trying 

In [56]:
df

,_time,_lineIndex,_lineLayer,_type,_cutDirection,_yCenter,_xCenter,_bpm
0,5.938,2,0,1,1,1.00,0.3,156
1,6.438,1,0,0,1,1.00,-0.3,156
2,6.938,3,1,1,5,1.55,0.9,156
3,7.938,2,0,1,1,1.00,0.3,156
4,8.438,1,2,0,0,2.10,-0.3,156
...,...,...,...,...,...,...,...,...
615,452.438,2,2,1,0,2.10,0.3,156
616,452.938,0,0,0,1,1.00,-0.9,156
617,453.438,1,0,1,6,1.00,-0.3,156
618,453.938,1,2,0,0,2.10,-0.3,156


In [57]:
left

,_time,_lineIndex,_lineLayer,_type,_cutDirection,_yCenter,_xCenter,_bpm,_xMovement,_yMovement,_totMovement,_angleChange,_timeChange,_seconds
1,6.438,1,0,0,1,1.00,-0.3,156,0.0,0.00,0.000000,NaN,0.0,0.000000
4,8.438,1,2,0,0,2.10,-0.3,156,0.0,1.10,1.100000,1.570796,2.0,0.769231
7,10.938,1,0,0,1,1.00,-0.3,156,0.0,-1.10,1.100000,-1.570796,2.5,0.961538
9,12.438,1,2,0,0,2.10,-0.3,156,0.0,1.10,1.100000,1.570796,1.5,0.576923
11,13.438,1,0,0,1,1.00,-0.3,156,0.0,-1.10,1.100000,-1.570796,1.0,0.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,448.938,2,0,0,7,1.00,0.3,156,0.6,-1.10,1.252996,-1.071450,1.0,0.384615
614,451.438,0,1,0,4,1.55,-0.9,156,-1.2,0.55,1.320038,-0.429762,2.5,0.961538
616,452.938,0,0,0,1,1.00,-0.9,156,0.0,-0.55,0.550000,-1.570796,1.5,0.576923
618,453.938,1,2,0,0,2.10,-0.3,156,0.6,1.10,1.252996,1.071450,1.0,0.384615


In [58]:
right

,_time,_lineIndex,_lineLayer,_type,_cutDirection,_yCenter,_xCenter,_bpm,_xMovement,_yMovement,_totMovement,_angleChange,_timeChange,_seconds
0,5.938,2,0,1,1,1.00,0.3,156,0.0,0.00,0.000000,NaN,0.00,0.000000
2,6.938,3,1,1,5,1.55,0.9,156,0.6,0.55,0.813941,0.741947,1.00,0.384615
3,7.938,2,0,1,1,1.00,0.3,156,-0.6,-0.55,0.813941,0.741947,1.00,0.384615
5,9.188,2,2,1,0,2.10,0.3,156,0.0,1.10,1.100000,1.570796,1.25,0.480769
6,10.438,2,0,1,1,1.00,0.3,156,0.0,-1.10,1.100000,-1.570796,1.25,0.480769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,446.438,1,0,1,6,1.00,-0.3,156,-0.6,-1.10,1.252996,1.071450,1.50,0.576923
610,447.438,3,1,1,5,1.55,0.9,156,1.2,0.55,1.320038,0.429762,1.00,0.384615
612,448.438,3,0,1,1,1.00,0.9,156,0.0,-0.55,0.550000,-1.570796,1.00,0.384615
615,452.438,2,2,1,0,2.10,0.3,156,-0.6,1.10,1.252996,-1.071450,4.00,1.538462


In [59]:
left_swings = len(left)
right_swings = len(right)
left_time = left['_seconds'].sum()
right_time = right['_seconds'].sum()







df['_timeChangeBefore'] = df.loc[:, ['_time']].diff().fillna(0)
df['_timeChangeBefore'] = (60 * df['_timeChangeBefore']) / df['_bpm']
df['_timeChangeAfter'] = abs(df.loc[:, ['_time']].diff(periods = -1).fillna(0))
df['_timeChangeAfter'] = (60 * df['_timeChangeAfter']) / df['_bpm']

minReactTimeBefore = float('inf')
minReactTimeAfter = float('inf')

firstType = df.loc[0, '_type']
secondType = df.loc[1, '_type']
secondLastType = df.loc[len(df) - 2, '_type']
lastType = df.loc[len(df) - 1, '_type']

if firstType == 3 & secondType != 3:
    minReactTimeAfter = df.loc[0, '_timeChangeAfter']

if lastType == 3 & secondLastType != 3:
    if (minReactTimeBefore == float('inf')) | (minReactTimeBefore > df.loc[len(df) - 1, '_timeChangeBefore']):
        minReactTimeBefore = df.loc[len(df) - 1, '_timeChangeBefore']

for i in range(len(df) - 2):
    previousType = df.loc[i, '_type']
    thisType = df.loc[i + 1, '_type']
    nextType = df.loc[i + 2, '_type']
    if thisType == 3:
        if previousType != 3:
            if (minReactTimeBefore == float('inf')) | (minReactTimeBefore > df.loc[i, '_timeChangeBefore']):
                minReactTimeBefore = df.loc[i, '_timeChangeBefore']
        if nextType != 3:
            if (minReactTimeAfter == float('inf')) | (minReactTimeAfter > df.loc[i, '_timeChangeAfter']):
                minReactTimeAfter = df.loc[i, '_timeChangeAfter']


right_avg_sps = right_swings / right_time
left_avg_sps = left_swings / left_time

In [60]:
print('The shortest reaction time before a bomb is ' + str(minReactTimeBefore) + ' seconds.')
print('The shortest reaction time after a bomb is ' + str(minReactTimeAfter) + ' seconds.')
print('The average right sps is ' + str(right_avg_sps) + ' swings per second.')
print('The average left sps is ' + str(left_avg_sps) + ' swings per second.')

The shortest reaction time before a bomb is 0.19230769230769232 seconds.
The shortest reaction time after a bomb is 0.0 seconds.
The average right sps is 1.551284916201117 swings per second.
The average left sps is 1.5130434782608693 swings per second.
